In [ ]:
!pip install simpletransformers emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import logging
import pandas as pd
import argparse
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
args = {'model_type' : 'bertweet',
        'tokenizer_name' : 'vinai/bertweet-base',
        'path_to_train' : '/content/drive/MyDrive/advNLP/twibot20/train.json',
        'path_to_val' : '/content/drive/MyDrive/advNLP/twibot20/val.json',
        'path_to_test' : '/content/drive/MyDrive/advNLP/twibot20/test.json',
        'num_train_epochs' : 10,
        'do_lower_case' : False,
        'early_stopping_consider_epochs' : False,
        'train_batch_size' : 128,
        'eval_batch_size': 128,
        'evaluate_during_training' : True,
        'learning_rate' : 4e-5, 
        'output_dir' : '/content/drive/MyDrive/advNLP/outputs/description/bertweet/',
        'overwrite_output_dir' : True,
        'manual_seed' : 11711,
        'use_early_stopping' : True,
        'early_stopping_metric' : 'eval_loss',
        'sliding_window' : False,
        'stride' : 0.8,
        'use_cuda' : True,
        'max_seq_length' : 128}

In [ ]:
def get_dataset(path_to_data):
  with open(path_to_data) as f:
    data = json.loads(f.read())
  data = [[x['description'], 1 if x['label']=='bot' else 0] for x in data]
  df = pd.DataFrame(data)
  df.columns = ["text", "labels"]
  return df

In [ ]:
def make_model(args):
  model_args = ClassificationArgs(  num_train_epochs=args['num_train_epochs'], 
                                    do_lower_case=args['do_lower_case'],
                                    early_stopping_consider_epochs=args['early_stopping_consider_epochs'],
                                    train_batch_size=args['train_batch_size'],
                                    eval_batch_size=args['eval_batch_size'],
                               	    evaluate_during_training=args['evaluate_during_training'],
                                    learning_rate=args['learning_rate'],
                                    output_dir=args['output_dir'],
                                    overwrite_output_dir=args['overwrite_output_dir'],
                                    manual_seed=args['manual_seed'],
                                    use_early_stopping=args['use_early_stopping'],
                                    early_stopping_metric=args['early_stopping_metric'],
                                    sliding_window=args['sliding_window'],
                                    stride=args['stride']
                                    
																		)
  model = ClassificationModel(args['model_type'], args['tokenizer_name'], 
                              args=model_args, use_cuda=args['use_cuda'])
  return model


def get_metrics(result):
  accuracy = (result['tp'] + result['tn']) / (result['tp'] + result['tn'] + result['fp'] + result['fn'])
  precision = result['tp'] / (result['tp'] + result['fp'])
  recall = result['tp'] / (result['tp'] + result['fn'])
  return {'acc' : accuracy, 'precision':precision, 'recall' : recall}

def train_test(args):
  logging.basicConfig(level=logging.ERROR)
  transformers_logger = logging.getLogger("transformers")
  transformers_logger.setLevel(logging.WARNING)
  
  train_data = get_dataset(args['path_to_train'])
  val_data = get_dataset(args['path_to_val'])
  test_data = get_dataset(args['path_to_test'])
  
  model = make_model(args)
  model.train_model(train_data, eval_df=val_data)
  
  result, model_outputs, wrong_predictions = model.eval_model(val_data)
  metrics = get_metrics(result)
  print("Validation results = ", metrics)
  
  result, model_outputs, wrong_predictions = model.eval_model(test_data)
  metrics = get_metrics(result)
  print("Test results = ", metrics)
  
  return model

In [ ]:
model = train_test(args)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

Validation results =  {'acc': 0.695137420718816, 'precision': 0.7114825581395349, 'recall': 0.7513430544896393}


  0%|          | 0/1183 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Test results =  {'acc': 0.7066779374471682, 'precision': 0.7138686131386861, 'recall': 0.7640625}


In [ ]:
model = ClassificationModel("bertweet", '/content/drive/MyDrive/advNLP/outputs/description/bertweet/')

In [ ]:
test_data = get_dataset(args['path_to_test'])
result, model_outputs, wrong_predictions = model.eval_model(test_data)

  0%|          | 0/1183 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
! pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 4.0 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=181916cebb6652ec9c52daf58c37c231e99dcbb160f77f287b62924afb74418c
  Stored in directory: /root/.cache/pip/wheels/e6/a6/20/cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime


In [ ]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import numpy as np

explainer = LimeTextExplainer(class_names=[0, 1])

output_probs = model.predict(list(test_data['text'].values))[1]
output_probs =  np.exp(output_probs)
output_probs = output_probs / np.sum(output_probs, axis=1, keepdims=True)

  0%|          | 0/1183 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
def predict_proba(x):
  logits = np.exp(model.predict(x)[1])
  return logits / np.sum(logits, axis=1, keepdims=True)

count = 0
test_explanations = []

for index, row in test_data.iterrows():

    bot_prob = output_probs[index, 1]
    pred = int(bot_prob>0.5)


    if len(row['text'].strip())==0:
      result = {"text" : row['text'], "label" : row['labels'],
            "weights" : [], "bot_prob" : bot_prob, "misclassified" : pred!=row['labels']}
    else:
      try:
        exp = explainer.explain_instance(row['text'], predict_proba, num_features=6)
        result = {"text" : row['text'], "label" : row['labels'],
              "weights" : exp.as_list(), "bot_prob" : bot_prob, "misclassified" : pred!=row['labels']}
      except:
        result = {"text" : row['text'], "label" : row['labels'],
            "weights" : [], "bot_prob" : bot_prob, "misclassified" : pred!=row['labels']}

    test_explanations.append(result)
    count += 1


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
test_explanations_df = pd.DataFrame.from_records(test_explanations)
test_explanations_df.to_csv('/content/drive/MyDrive/advNLP/twibot20/test_explanations.csv')

In [74]:
test_explanations_df

,text,label,weights,bot_prob,misclassified
0,We're a father/son duo sharing the love we pos...,1,"[(follow, 0.0969019890645695), (cards, 0.09435...",0.988799,False
1,,1,[],0.698638,False
2,,1,[],0.698638,False
3,Satirizing the sports card industry one tweet ...,1,"[(card, 0.05908220623257466), (CardboardChaos,...",0.978385,False
4,Texas Tech professor of human devt and family ...,1,"[(grad, 0.2434389144886219), (Greatest, -0.226...",0.314945,True
...,...,...,...,...,...
96,"wife,mom,gma,Dem,liberal,truth,justice,equalit...",1,"[(liberal, 0.0038273853515450843), (singlePaye...",0.983816,False
97,"Mother of 4, Wife, Aunt,Sister,Friend,Public S...",1,"[(BidenHarris2020, 0.0038983825303152224), (Mo...",0.992139,False
98,Singer l Activist l Mom l Vegan l Wife l Creat...,0,"[(WeNeedToTalk, -0.3066512521278763), (Singer,...",0.080357,False
99,was laurablonsky.#TheResistance #NotMyPresiden...,1,"[(laurablonsky, 0.0030197516533927663), (lefti...",0.984767,False


In [ ]:
all_weights = [] 
for y in test_explanations_df['weights'].values:
  all_weights = all_weights + list(y)
all_weights.sort(key = lambda x: -abs(x[1]))

In [ ]:
all_weights[:10]

[('Photographer', 0.5110523457209584),
 ('travel', 0.4685751334933018),
 ('Indiana', 0.4546551566749081),
 ('ScissorGendered', 0.45009460271376656),
 ('faint', -0.41803550528369654),
 ('being', 0.3949726610632242),
 ('LOVE', -0.38423510945956246),
 ('𝐌𝐎𝐌𝐎', 0.3609781054244193),
 ('Blue', 0.34958554443122025),
 ('parkeerboete', 0.34297538533752986)]